# APPM 6640 - Homework 4
***
This assignment is due on D2L by 11:59pm on **Thursday, July 15th**. Your solutions to the written exercises should be completed in Markdown directly in this Jupyter Notebook.  Ideally, your solutions to the programming problems should be written directly in the notebook as well, but if you like you can write your functions in a $\texttt{.jl}$ script and then load your script into the notebook.  

### Problem 1: Two-Dimensional Five-Point Stencil 
When the two dimensional model problem is discretized on a uniform grid with $h_x = h_y = h$, the coefficients at each grid point are given by the five-point stencil 

$$
A^h = \frac{1}{h^2} \left( 
\begin{array}{ccc}
 & -1 & \\
 -1 & 4 & -1 \\
  & -1 & 
\end{array}
\right)
$$

What does the stencil for the Galerkin coarse-grid operator, $A^{2h} = RA^hP$, look like if $P$ is based on bilinear interpolation and $R$ is based on (a) full-weighting and (b) injection? 

### Problem 2: Two-Dimensional Nine-Point Stencil 
Repeat the previous problem with the nine-point stencil arising from a Galerkin finite-element discretization of the 2D model problem: 

$$
A^h = \frac{1}{3h^2} \left( 
\begin{array}{ccc}
-1 & -1 & -1 \\
-1 &  8 & -1 \\
-1 & -1 & -1
\end{array}
\right)
$$


### Problem 3: Two-Dimensional Nine-Point Stencil 
Show (mathematically) that the 2D bilinear interpolation operator $P$ can be written as 

$$
P = P_y \otimes P_x 
$$

where $P_x$ and $P_y$ are the 1D linear interpolation operators associated with the grid lines of constant $y$ and constant $x$, respectively. 

### Problem 4: Bilinear Interpolation Operator  
Write a function $\texttt{bilinterp}$ with the following API that accepts integers $L_x$ and $L_y$ and returns the bilinear interpolation operator $P$ of type $\texttt{SparseMatrixCSC}$ that interpolates vectors ${\bf v}^{2h}$ to vectors ${\bf v}^h$ where ${\bf v}^h$ lives on a grid with $(2^{L_x}-1)(2^{L_y}-1)$ interior grid points.  

In [ ]:
function bilinterp(Lx::Int64, Ly::Int64)

### Problem 5: Full-Weighting Restriction Operator  
Write a function $\texttt{fullweighting}$ with the following API that accepts integers $L_x$ and $L_y$ and returns the 2D full-weighting restriction operator $R$ of type $\texttt{SparseMatrixCSC}$ that restricts vectors ${\bf v}^{h}$ to vectors ${\bf v}^{2h}$ where ${\bf v}^h$ lives on a grid with $(2^{L_x}-1)(2^{L_y}-1)$ interior grid points.  

In [ ]:
function fullweighting(Lx::Int64, Ly::Int64)

### Problem 6: Level Hierarchy 
Recall the level type data structure discussed in class. 

In [ ]:
type level
    A::AbstractMatrix
    P::AbstractMatrix
    R::AbstractMatrix
    v::AbstractVector
    f::AbstractVector
    smoother::Function
    level(A,P,R,v,f,s) = new(copy(A), copy(P), copy(R), copy(v), copy(f), s)
end

Write a function $\texttt{MGInitialize2D}$ with the following API that accepts an integers $\texttt{fineLx}$  and $\texttt{fineLy}$ representing the resolution on the finest level,  an integer $\texttt{numLevels}$ representing the total number of levels in the hierarchy, a float $\sigma$, and a function $\texttt{smoother}$ indicating the relaxation technique to use and returns an array of level types initialized with $A$, $P$, $R$, ${\bf v}$ and ${\bf f}$ of appropriate initializations and sizes. 

**Notes**: 
- The coarse grid operators $A$ should be constructed by rediscretizing the model problem on the appropriate grid level. 
- You are free to add optional default parameters to the end of the API that make your life easier, provided that the default function call does something reasonable. 

In [ ]:
function MGInitialize2D(fineLx::Int64, fineLy::Int64, numLevels::Int64, sigma::Float64, smoother::Function)

### Problem 7: FMG-Cycle 
If you wrote robust code in Homework 3 then your $\texttt{VCycle!}$ function work out-of-the-box on the new 2D hierarchy.  Your first task is to do some testing to check that this is the case.  Once you're confident that your $\texttt{VCycle!}$ works, write a new function $\texttt{FMGCycle!}$ with the following API that accepts an array of level types representing the multigrid hierarchy, integers $\texttt{nu1}$ and $\texttt{nu2}$ representing the number of pre- and post-relaxation sweeps and performs a single FMG-Cycle (concluding the single V-Cycle starting from the finest grid). The optional integer $\texttt{myLev}$ indicating the current level in the hierarchy will be helpful for recursive implementations. 

In [ ]:
function FMGCycle!(lv::Array{level,1},nu1::Int64,nu2::Int64,myLev::Int64=1)

### Problem 8: Experiments and Analysis

Test the V-Cycle and FMG-Cycle codes separately on the 2D Model Problem with a known exact solution.  First try $f(x,y) = 0$ (with solution $u(x,y) = 0$) and then try $f(x,y) = C\sin(k\pi x)\sin(\ell \pi y)$ where $C$ is real and $k$ and $\ell$ are integer constants.  The exact solution to this problem is given by 

$$
u(x,y) = \frac{C}{\pi^2k^2 + \pi^2\ell^2 + \sigma} \sin{(k\pi x)} \sin{(\ell \pi y)}. 
$$

For a complete analysis of FMG, check that your cycle is converging to the level of discretization error after a single complete FMG-cycle. 